In [9]:
from dataclasses import dataclass

from langchain.agents import create_agent
from langchain_ollama import ChatOllama
model=ChatOllama(model="gpt-oss:120b-cloud")  

@dataclass
class Context:
    user_name: str

agent = create_agent(
    model=model,
    tools=[],
    context_schema=Context  
)

agent.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
    context=Context(user_name="John Smith")  
)

{'messages': [HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='bfbbee17-af73-47b9-9595-a9504c07af1d'),
  AIMessage(content='I’m sorry, but I don’t have any information about your name. If you’d like to share it, feel free to let me know!', additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b-cloud', 'created_at': '2025-10-17T02:34:16.079093438Z', 'done': True, 'done_reason': 'stop', 'total_duration': 778127494, 'load_duration': None, 'prompt_eval_count': 76, 'prompt_eval_duration': None, 'eval_count': 90, 'eval_duration': None, 'model_name': 'gpt-oss:120b-cloud', 'model_provider': 'ollama'}, id='lc_run--29faaec9-6646-4cde-889f-21f661271bd2-0', usage_metadata={'input_tokens': 76, 'output_tokens': 90, 'total_tokens': 166})]}

inside tools

In [7]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime  

@dataclass
class Context:
    user_id: str

@tool
def fetch_user_email_preferences(runtime: ToolRuntime[Context]) -> str:  
    """Fetch the user's email preferences from the store."""
    user_id = runtime.context.user_id  

    preferences: str = "The user prefers you to write a brief and polite email."
    if runtime.store:  
        if memory := runtime.store.get(("users",), user_id):  
            preferences = memory.value["preferences"]

    return preferences

ImportError: cannot import name 'ToolRuntime' from 'langchain.tools' (d:\SourceCode-2\langchain-alpha1\.venv\Lib\site-packages\langchain\tools\__init__.py)

inside middleware

In [ ]:
from dataclasses import dataclass

from langchain.messages import AnyMessage
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import dynamic_prompt, ModelRequest, before_model, after_model
from langgraph.runtime import Runtime
from langchain_ollama import ChatOllama
model=ChatOllama(model="gpt-oss:120b-cloud")
@dataclass
class Context:
    user_name: str

# Dynamic prompts
@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    user_name = request.runtime.context.user_name  
    system_prompt = f"You are a helpful assistant. Address the user as {user_name}."
    return system_prompt

# Before model hook
@before_model
def log_before_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:  
    print(f"Processing request for user: {runtime.context.user_name}")  
    return None

# After model hook
@after_model
def log_after_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:  
    print(f"Completed request for user: {runtime.context.user_name}")  
    return None

agent = create_agent(
    model=model,
    tools=[],
    middleware=[dynamic_system_prompt, log_before_model, log_after_model],  
    context_schema=Context
)

agent.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
    context=Context(user_name="John Smith")
)

Processing request for user: John Smith
Completed request for user: John Smith


{'messages': [HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='c238e61d-7b2e-499b-ac9f-587657ed6c51'),
  AIMessage(content='I don’t have any information about your name. If you’d like me to refer to you by a particular name, just let me know!', additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b-cloud', 'created_at': '2025-10-17T02:34:46.060798017Z', 'done': True, 'done_reason': 'stop', 'total_duration': 834406190, 'load_duration': None, 'prompt_eval_count': 76, 'prompt_eval_duration': None, 'eval_count': 75, 'eval_duration': None, 'model_name': 'gpt-oss:120b-cloud', 'model_provider': 'ollama'}, id='lc_run--7b8b80d1-06eb-4f1a-9720-ff02b8a80160-0', usage_metadata={'input_tokens': 76, 'output_tokens': 75, 'total_tokens': 151})]}